In [118]:
import os
os.sys.path.append('..')
%reload_ext autoreload
%autoreload 2


In [119]:
from train import *
from data import format_thing, get_id_for_comments, thread2tree, get_dataset

In [125]:
parser = ArgumentParser()
parser.add_argument("--dataset_path", type=str, default="../data/reddit_threads/", help="Path or url of the dataset. If empty download from S3.")
parser.add_argument("--dataset_cache", type=str, default='./dataset_cache', help="Path or url of the dataset cache")
parser.add_argument("--model_checkpoint", type=str, default="openai-gpt", help="Path, url or short name of the model")
parser.add_argument("--num_candidates", type=int, default=2, help="Number of candidates for training")
parser.add_argument("--max_history", type=int, default=2, help="Number of previous exchanges to keep in history")
parser.add_argument("--train_batch_size", type=int, default=4, help="Batch size for training")
parser.add_argument("--valid_batch_size", type=int, default=4, help="Batch size for validation")
parser.add_argument("--gradient_accumulation_steps", type=int, default=8, help="Accumulate gradients on several steps")
parser.add_argument("--lr", type=float, default=6.25e-5, help="Learning rate")
parser.add_argument("--lm_coef", type=float, default=1.0, help="LM loss coefficient")
parser.add_argument("--mc_coef", type=float, default=1.0, help="Multiple-choice loss coefficient")
parser.add_argument("--max_norm", type=float, default=1.0, help="Clipping gradient norm")
parser.add_argument("--n_epochs", type=int, default=3, help="Number of training epochs")
parser.add_argument("--personality_permutations", type=int, default=1, help="Number of permutations of personality sentences")
parser.add_argument("--eval_before_start", action='store_true', help="If true start with a first evaluation before training")
parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu", help="Device (cuda or cpu)")
parser.add_argument("--fp16", type=str, default="", help="Set to O0, O1, O2 or O3 for fp16 training (see apex documentation)")
parser.add_argument("--local_rank", type=int, default=-1, help="Local rank for distributed training (-1: not distributed)")


_StoreAction(option_strings=['--local_rank'], dest='local_rank', nargs=None, const=None, default=-1, type=<class 'int'>, choices=None, help='Local rank for distributed training (-1: not distributed)', metavar=None)

In [126]:
args = parser.parse_args([])

In [127]:
# logging is set to INFO (resp. WARN) for main (resp. auxiliary) process. logger.info => log main process only, logger.warning => log all processes
logging.basicConfig(level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN)
logger.warning("Running process %d", args.local_rank)  # This is a logger.warning: it will be printed by all distributed processes
logger.info("Arguments: %s", pformat(args))

INFO:../train.py:Arguments: Namespace(dataset_cache='./dataset_cache', dataset_path='../data/reddit_threads/', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='openai-gpt', n_epochs=3, num_candidates=2, personality_permutations=1, train_batch_size=4, valid_batch_size=4)


# Check dataloading why so few

In [150]:
logger.info("Prepare tokenizer, pretrained model and optimizer - add special tokens for fine-tuning")
tokenizer_class = GPT2Tokenizer if "gpt2" in args.model_checkpoint else OpenAIGPTTokenizer
tokenizer = tokenizer_class.from_pretrained(args.model_checkpoint)
tokenizer.set_special_tokens(SPECIAL_TOKENS)
# model.set_num_special_tokens(len(SPECIAL_TOKENS))

INFO:../train.py:Prepare tokenizer, pretrained model and optimizer - add special tokens for fine-tuning
INFO:pytorch_pretrained_bert.tokenization_openai:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/openai-gpt-vocab.json from cache at /home/wassname/.cache/torch/pytorch_pretrained_bert/4ab93d0cd78ae80e746c27c9cd34e90b470abdabe0590c9ec742df61625ba310.b9628f6fe5519626534b82ce7ec72b22ce0ae79550325f45c604a25c0ad87fd6
INFO:pytorch_pretrained_bert.tokenization_openai:loading merges file https://s3.amazonaws.com/models.huggingface.co/bert/openai-gpt-merges.txt from cache at /home/wassname/.cache/torch/pytorch_pretrained_bert/0f8de0dbd6a2bb6bde7d758f4c120dd6dd20b46f2bf0a47bc899c89f46532fde.20808570f9a3169212a577f819c845330da870aeb14c40f7319819fce10c3b76
INFO:pytorch_pretrained_bert.tokenization_openai:Special tokens {'<bos>': 40478, '<eos>': 40479, '<speaker1>': 40480, '<speaker2>': 40481, '<pad>': 40482}


In [155]:
personachat = get_dataset(tokenizer, args.dataset_path)

test: 100%|██████████| 245/245 [00:01<00:00, 235.79file/s]
INFO:../data.py:Tokenize and encode the dataset


In [278]:
personachat

KeyboardInterrupt: 

In [162]:
[[len(s) for s in c['candidates']] for c in utterances]

[[220, 117, 67],
 [78, 43, 35],
 [39, 98, 75],
 [37, 112, 86],
 [107, 40, 80],
 [112, 308, 55],
 [58, 48, 44],
 [100, 28, 98],
 [38, 41, 333],
 [42, 35, 45],
 [39, 72, 44],
 [68, 72, 85],
 [36, 51, 56],
 [119, 71, 50],
 [42, 119, 73],
 [136, 70, 34],
 [59, 212, 26],
 [503, 209, 206],
 [92, 90, 79],
 [37, 25, 272],
 [75, 330, 68],
 [41, 40, 512],
 [49, 69, 75],
 [64, 73, 29],
 [90, 129, 36],
 [55, 141, 28],
 [37, 66, 126],
 [46, 34, 110],
 [76, 35, 26],
 [46, 27, 37],
 [43, 33, 34],
 [49, 105, 56],
 [91, 82, 80],
 [70, 34, 35],
 [82, 77, 151],
 [316, 28, 28],
 [512, 35, 61],
 [34, 50, 36],
 [121, 154, 49],
 [46, 114, 31],
 [207, 48, 116],
 [83, 37, 40],
 [121, 139, 24],
 [42, 41, 288],
 [50, 29, 26],
 [119, 103, 145],
 [54, 39, 50],
 [40, 94, 80],
 [40, 32, 69],
 [40, 62, 49],
 [64, 30, 26],
 [92, 34, 31],
 [132, 31, 32],
 [71, 51, 64],
 [101, 99, 98],
 [142, 64, 150],
 [152, 97, 282],
 [122, 193, 64],
 [263, 94, 49],
 [177, 52, 69],
 [51, 139, 107],
 [58, 50, 51],
 [60, 69, 27],
 [108,

In [220]:
utterances = personachat['train'][0]['utterances']
[len(c['candidates']) for c in utterances]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [221]:
logger.info("Build inputs and labels")
datasets = {"train": defaultdict(list), "valid": defaultdict(list), "test": defaultdict(list)}
for dataset_name, dataset in personachat.items():
    num_candidates = len(dataset[0]["utterances"][0]["candidates"])
    if args.num_candidates > 0 and dataset_name == 'train':
        num_candidates = min(args.num_candidates, num_candidates)
    for dialog in dataset:
        persona = dialog["personality"].copy()
        for _ in range(args.personality_permutations):
            for utterance in dialog["utterances"]:
                history = utterance["history"][-(2*args.max_history+1):]
                for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
                    lm_labels = bool(j == num_candidates-1)
                    instance, _ = build_input_from_segments(persona, history, candidate, tokenizer, lm_labels)
                    for input_name, input_array in instance.items():
                        datasets[dataset_name][input_name].append(input_array)
                datasets[dataset_name]["mc_labels"].append(num_candidates - 1)
                datasets[dataset_name]["n_candidates"] = num_candidates
            persona = [persona[-1]] + persona[:-1]  # permuted personalities

INFO:../train.py:Build inputs and labels


AssertionError: input should be less than max len

In [252]:
logger.info("Build inputs and labels")
datasets = {"train": defaultdict(list), "valid": defaultdict(list), "test": defaultdict(list)}
for dataset_name, dataset in personachat.items():
    num_candidates = len(dataset[0]["utterances"][0]["candidates"])
    if args.num_candidates > 0 and dataset_name == 'train':
        num_candidates = min(args.num_candidates, num_candidates)
    for dialog in dataset:
        persona = dialog["personality"].copy()
        for _ in range(args.personality_permutations):
            for utterance in dialog["utterances"]:
                history = utterance["history"][-(2*args.max_history+1):]
                for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
                    lm_labels = bool(j == num_candidates-1)
#                     instance, _ = build_input_from_segments(persona, history, candidate, tokenizer, lm_labels)
#                     for input_name, input_array in instance.items():
#                         datasets[dataset_name][input_name].append(input_array)
#                 datasets[dataset_name]["mc_labels"].append(num_candidates - 1)
#                 datasets[dataset_name]["n_candidates"] = num_candidates
#             persona = [persona[-1]] + persona[:-1]  # permuted personalities

INFO:../train.py:Build inputs and labels


In [230]:
def _truncate_seq_pair_n(tokens, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    # from https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_classifier_dataset_utils.py#L482
    while True:
        total_length = sum(len(s) for s in tokens)
        if total_length <= max_length:
            break
        longest = sorted(tokens, key=len, reverse=True)[0]
        longest.pop()

In [258]:
# Now look at the build input to find where we should clip
reply=candidate
with_eos=True
# build_input_from_segments(persona, history, candidate, tokenizer, lm_labels)
bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])

tokenizer.max_len

instance = {}
sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])]
sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
print(sequence)
sequence2 = sequence

instance["input_ids"] = list(chain(*sequence))
instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
instance["mc_token_ids"] = len(instance["input_ids"]) - 1
instance["lm_labels"] = [-1] * len(instance["input_ids"])
if lm_labels:
    instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
    
assert len(instance["input_ids"]) < tokenizer.max_len

[[40478, 3260], [40481, 269, 269, 269, 269, 252, 40477, 28168, 271, 11240, 16260, 492, 970, 1, 507, 278, 13, 24, 21, 9, 53, 12, 53, 17, 12, 50, 53, 43, 20343, 28, 24, 268, 40477, 616, 1120, 925, 510, 6907, 40477, 269, 269, 269, 269, 514, 15925, 13, 44, 265], [40480, 269, 269, 269, 269, 241, 15925, 13, 44, 265, 40477, 1288, 640, 481, 3269, 249, 925, 669, 520, 8955, 510, 239, 40477, 269, 269, 269, 269, 590, 2151, 21, 2041, 14, 251], [40481, 269, 269, 269, 269, 255, 2151, 21, 2041, 14, 251, 40477, 912, 520, 899, 551, 485, 580, 487, 40477, 269, 269, 269, 269, 483, 2151, 21, 30, 712, 268], [40480, 269, 269, 269, 269, 255, 2151, 21, 30, 712, 268, 40477, 568, 507, 942, 773, 40477, 269, 269, 269, 269, 483, 2151, 21, 23437, 261, 40479]]


In [274]:
len(history), len(sequence)
tokenizer.max_len

(3, 5)

In [277]:
instance = {}
sequence = [list(chain(*persona))] + history + [reply ]

# clip the chat history, by removing from the longest message
_truncate_seq_pair_n(sequence, tokenizer.max_len - len(history) - 2 - with_eos)
persona_c = sequence[0]
history_c = sequence[1:-1]
reply_c = sequence[-1]

# add tokens
sequence = [[bos] + persona_c] + history_c + [reply_c + ([eos] if with_eos else [])]
print(sequence)
# add speaker tokens
sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
print(sequence)

[[40478, 3260], [269, 269, 269, 269, 252, 40477, 28168, 271, 11240, 16260, 492, 970, 1, 507, 278, 13, 24, 21, 9, 53, 12, 53, 17], [269, 269, 269, 269, 241, 15925, 13, 44, 265, 40477, 1288, 640, 481, 3269, 249, 925, 669, 520, 8955, 510, 239, 40477, 269], [269, 269, 269, 269, 255, 2151, 21, 2041, 14, 251, 40477, 912, 520, 899, 551, 485, 580, 487, 40477, 269, 269, 269, 269], [269, 269, 269, 269, 255, 2151, 21, 30, 712, 268, 40477, 568, 507, 942, 773, 40477, 269, 269, 269, 269, 483, 2151, 21, 23437, 40479]]
[[40478, 3260], [40481, 269, 269, 269, 269, 252, 40477, 28168, 271, 11240, 16260, 492, 970, 1, 507, 278, 13, 24, 21, 9, 53, 12, 53, 17], [40480, 269, 269, 269, 269, 241, 15925, 13, 44, 265, 40477, 1288, 640, 481, 3269, 249, 925, 669, 520, 8955, 510, 239, 40477, 269], [40481, 269, 269, 269, 269, 255, 2151, 21, 2041, 14, 251, 40477, 912, 520, 899, 551, 485, 580, 487, 40477, 269, 269, 269, 269], [40480, 269, 269, 269, 269, 255, 2151, 21, 30, 712, 268, 40477, 568, 507, 942, 773, 40477, 269,

In [276]:
ll = [len(s) for s in sequence]
ll, sum(ll)

([2, 24, 24, 24, 26], 100)

In [255]:
a =[[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])]
[len(s) for s in a]

[2, 47, 32, 29, 26]

In [174]:
len(instance["input_ids"]), len(instance["token_type_ids"]), len(instance["lm_labels"])

(140, 140, 140)

In [192]:
remaining_len = tokenizer.max_len - len(list(chain(*persona))) - 3

In [188]:
len(reply), len(list(chain(*history)))

(25, 108)

In [ ]:
logger.info("Pad inputs and convert to Tensor")
tensor_datasets = {"train": [], "valid": [], "test": []}
for dataset_name, dataset in datasets.items():
    dataset = pad_dataset(dataset, padding=tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
    for input_name in MODEL_INPUTS:
        tensor = torch.tensor(dataset[input_name])
        if input_name != "mc_labels":
            tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
        tensor_datasets[dataset_name].append(tensor)

In [ ]:
logger.info("Build train and validation dataloaders")
train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset) if args.distributed else None
valid_sampler = torch.utils.data.distributed.DistributedSampler(valid_dataset) if args.distributed else None
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, shuffle=(not args.distributed))
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.valid_batch_size, shuffle=False)

logger.info("Train dataset (Batch, Candidates, Seq length): {}. Batches: {}".format(train_dataset.tensors[0].shape, len(train_dataset)))
logger.info("Valid dataset (Batch, Candidates, Seq length): {}, Batches: {}".format(valid_dataset.tensors[0].shape, len(valid_dataset)))

# Other

In [99]:
# train_loader, val_loader, train_sampler, valid_sampler = get_data_loaders(args, tokenizer)

In [100]:
# from utils import PERSONACHAT_URL, HF_FINETUNED_MODEL, cached_path
# dataset_path=args.dataset_path
# dataset_cache=args.dataset_cache
# dataset_path = dataset_path or PERSONACHAT_URL
# dataset_cache = dataset_cache + '_' + type(tokenizer).__name__  # Do avoid using GPT cache for GPT-2 and vice-versa

# personachat_file = cached_path(dataset_path)
# with open(personachat_file, "r", encoding="utf-8") as f:
#     dataset = json.loads(f.read())

# helpers

In [105]:
# def get_dataset(tokenizer, dataset_path, dataset_cache=None):
#     """ Get PERSONACHAT from S3 """
#     dataset_path = dataset_path or PERSONACHAT_URL
#     dataset_cache = dataset_cache + '_' + type(tokenizer).__name__  # Do avoid using GPT cache for GPT-2 and vice-versa
#     if dataset_cache and os.path.isfile(dataset_cache):
#         logger.info("Load tokenized dataset from cache at %s", dataset_cache)
#         dataset = torch.load(dataset_cache)
#     else:
#         logger.info("Download dataset from %s", dataset_path)
#         personachat_file = cached_path(dataset_path)
#         with open(personachat_file, "r", encoding="utf-8") as f:
#             dataset = json.loads(f.read())

#         logger.info("Tokenize and encode the dataset")
#         def tokenize(obj):
#             if isinstance(obj, str):
#                 return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
#             if isinstance(obj, dict):
#                 return dict((n, tokenize(o)) for n, o in obj.items())
#             return list(tokenize(o) for o in obj)
#         dataset = tokenize(dataset)
#         if dataset_cache:
#             torch.save(dataset, dataset_cache)
#     return dataset

In [106]:
# # def get_data_loaders(args, tokenizer):
# """ Prepare the dataset for training and evaluation """
# personachat = get_dataset(tokenizer, args.dataset_path, args.dataset_cache)

In [107]:
# print(personachat.keys(), personachat['train'][0].keys())
# """
# We can pass in history as previous comments
# persona... none. Or OP
# speaker... yeah leave it as 1,2,1,2 for now
# so all it's doing it putting it in order
# """
# print('persona', [tokenizer.decode(p) for p in persona])
# print('history', [tokenizer.decode(p) for p in history])
# print('candidate', tokenizer.decode(candidate))

In [117]:
len(list(chain(*personalities.values())))

170

In [108]:
# # Load reddit comments from pickles
# from pathlib import Path
# import pickle
# data_dir = Path('../data/reddit_threads/')
# subreddit_paths = [d for d in data_dir.glob('*/') if d.is_dir()]
# subreddit_path = subreddit_paths[0]
# # print(subreddit_path)
# subreddit_files = sorted(subreddit_path.glob('*.pickle'))
# print(len(subreddit_files))

In [109]:
# import itertools
# import copy
# import collections
        
# def get_id_for_comments(thing):
#     if thing["type"] == "submission":
#         return "t3_" + thing["id"]
#     else:
#         return "t1_" + thing["id"]
    
# def thread2queue(comment_dict, submission):
#     # Now we want to reconstruct the comment heirachy.
#     # 0. Init with the submission in the queue. start with this as target
#     # 1. Look at target item in the queue, find it's top rated child comment
#     #  1a. If it has one, pop it out, put it at end of queue, go to 1
#     #  1b. If it doesn't have comment left, go to previous item in queue
#     comment_dict = copy.deepcopy(comment_dict)
#     queue = [submission]
#     submission_id = get_id_for_comments(submission)
#     while len(list(itertools.chain(*comment_dict.values()))) > 0:
#         for queue_position in range(len(queue) - 1, -1, -1):
#             current_id = get_id_for_comments(queue[queue_position])
#             found = comment_dict[current_id]
#             if len(found):
#                 break
#         next_comment = comment_dict[current_id].pop()
#         queue.append(next_comment)

#     # now format
#     return queue

# def format_thing(thing, submission_id, tokenizer):
#     """Format a submission or comment using special tokens and tokens"""
#     def tokenize(obj):
#         return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
#     bos, eos, speaker1, speaker2, pad, S, ES, R, ER, T, ET = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
#     if thing["type"] == "submission":
#         return (
#             [S]
#             + tokenize("\n".join([thing["url"], thing["title"], thing["selftext"]]))
#             + [ES ]
#             + tokenize(thing["id"])
#         )
#     elif thing["parent_id"] == submission_id:
#         return (
#             [T]
#             + tokenize(thing["parent_id"][3:] + "\n")            
#             + tokenize(thing["body"])
#             + [ET]
#             + tokenize(thing["id"])
#         )
#     else:
#         return (
#             [R]
#             + tokenize(thing["parent_id"][3:]
#             + "\n"
#             + thing["body"])
#             + [ER]
#             + tokenize(thing["id"])
#         )



In [110]:
# def build_input_from_segments(thread, tokenizer, lm_labels=False, with_eos=True):
#     """ Build a sequence of input from 3 segments: persona, history and last reply """
#     bos, eos, speaker1, speaker2, pad, S, ES, R, ER, T, ET = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
    
#     # So build sequence from reddit comments
#     queue = thread2queue(thread['comment_dict'], thread['submission'])
#     submission_id = get_id_for_comments(queue[0])
#     queue_tokens = [format_thing(thing, submission_id, tokenizer) for thing in queue]
#     queue_tokens = list(itertools.chain(*queue_tokens))

#     instance = {}
#     sequence = [bos] + queue_tokens + ([eos] if with_eos else [])

#     instance["input_ids"] = sequence
#     instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence)]
#     instance["mc_token_ids"] = len(instance["input_ids"]) - 1
#     instance["lm_labels"] = [-1] * len(instance["input_ids"])
#     if lm_labels:
#         instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
#     return instance, sequence

In [28]:
# thread = pickle.load(subreddit_files[10].open('rb'))
# instance, sequence = build_input_from_segments(thread, tokenizer, lm_labels=False)
# print(tokenizer.decode(instance['input_ids']))

# Thread to persona json

In [111]:
dataset2 = get_dataset(tokenizer, '../data/reddit_threads/')

../data/reddit_threads/singularity/t3_9r9hpa.pickle pop from empty list
../data/reddit_threads/singularity/t3_9j3ciq.pickle pop from empty list


../data/reddit_threads/totallynotrobots/t3_8yrj86.pickle pop from empty list
../data/reddit_threads/totallynotrobots/t3_9tp6nq.pickle pop from empty list
../data/reddit_threads/totallynotrobots/t3_947zsn.pickle pop from empty list


../data/reddit_threads/aww/t3_camn17.pickle pop from empty list


../data/reddit_threads/totallynotrobots/t3_9zbfqf.pickle pop from empty list


INFO:../data.py:Tokenize and encode the dataset


In [66]:

def build_input_from_segments(persona, history, reply, tokenizer, lm_labels=False, with_eos=True):
    """ Build a sequence of input from 3 segments: persona, history and last reply """
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])

    instance = {}
    sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])]
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]

    instance["input_ids"] = list(chain(*sequence))
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-1] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
    return instance, sequence

In [68]:
logger.info("Build inputs and labels")
datasets = {"train": defaultdict(list), "valid": defaultdict(list), "test": defaultdict(list)}
for dataset_name, dataset in personachat.items():
    num_candidates = len(dataset[0]["utterances"][0]["candidates"])
    if args.num_candidates > 0 and dataset_name == 'train':
        num_candidates = min(args.num_candidates, num_candidates)
    for dialog in dataset:
        persona = dialog["personality"].copy()
        for _ in range(args.personality_permutations):
            for utterance in dialog["utterances"]:
                history = utterance["history"][-(2*args.max_history+1):]
                for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
                    lm_labels = bool(j == num_candidates-1)
                    instance, _ = build_input_from_segments(persona, history, candidate, tokenizer, lm_labels)
                    for input_name, input_array in instance.items():
                        datasets[dataset_name][input_name].append(input_array)
                datasets[dataset_name]["mc_labels"].append(num_candidates - 1)
                datasets[dataset_name]["n_candidates"] = num_candidates
            persona = [persona[-1]] + persona[:-1]  # permuted personalities

INFO:../train.py:Build inputs and labels


# Other

In [69]:
print('keys', datasets['train'].keys())
for key in datasets['train'].keys():
    d = datasets['train'][key]
    if isinstance(d, list):
        print(key, d[:1])
    else: print(key, d)

keys dict_keys(['input_ids', 'token_type_ids', 'mc_token_ids', 'lm_labels', 'mc_labels', 'n_candidates'])
input_ids [[40478, 17574, 22863, 40481, 269, 269, 269, 269, 255, 26281, 44, 559, 265, 512, 759, 256, 241, 3394, 12975, 244, 485, 728, 11101, 240, 485, 1909, 240, 488, 485, 4571, 244, 485, 8888, 239, 645, 600, 759, 256, 241, 6920, 649, 768, 240, 895, 636, 249, 1533, 257, 512, 604, 485, 1168, 688, 481, 3371, 485, 6920, 562, 688, 485, 1629, 6920, 239, 895, 636, 249, 1441, 512, 645, 512, 972, 510, 512, 825, 600, 604, 6417, 257, 249, 759, 256, 241, 580, 500, 754, 1082, 485, 699, 525, 239, 249, 727, 599, 512, 640, 1887, 240, 249, 976, 587, 240, 568, 249, 4484, 485, 1441, 525, 3555, 759, 256, 241, 580, 6787, 485, 3366, 240, 507, 28989, 247, 256, 241, 1621, 645, 600, 604, 6804, 522, 595, 239, 249, 1048, 2805, 525, 616, 759, 580, 1295, 557, 8238, 8103, 239, 3555, 1550, 7692, 4569, 247, 256, 241, 7750, 768, 240, 525, 636, 580, 547, 1579, 239, 249, 1048, 668, 1565, 670, 599, 256, 252, 1198, 5

In [70]:
logger.info("Pad inputs and convert to Tensor")
tensor_datasets = {"train": [], "valid": []}
for dataset_name, dataset in datasets.items():
    dataset = pad_dataset(dataset, padding=tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
    for input_name in MODEL_INPUTS:
        tensor = torch.tensor(dataset[input_name])
        if input_name != "mc_labels":
            tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
        tensor_datasets[dataset_name].append(tensor)



INFO:../train.py:Pad inputs and convert to Tensor


RuntimeError: shape '[-1, 3, 1375]' is invalid for input of size 413875

In [ ]:
logger.info("Build train and validation dataloaders")
train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset) if args.distributed else None
valid_sampler = torch.utils.data.distributed.DistributedSampler(valid_dataset) if args.distributed else None
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, shuffle=(not args.distributed))
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.valid_batch_size, shuffle=False)

logger.info("Train dataset (Batch, Candidates, Seq length): {}".format(train_dataset.tensors[0].shape))
logger.info("Valid dataset (Batch, Candidates, Seq length): {}".format(valid_dataset.tensors[0].shape))
train_loader, valid_loader, train_sampler, valid_sampler